In [1]:
import torch

from pytorch_metric_learning.distances import LpDistance, CosineSimilarity, DotProductSimilarity
from pytorch_metric_learning.losses import NTXentLoss, SupConLoss
from pytorch_metric_learning.reducers import AvgNonZeroReducer, PerAnchorReducer
from pytorch_metric_learning.utils import common_functions as c_f

import os

TEST_DEVICE = torch.device(os.environ.get("TEST_DEVICE", "cuda"))

dtypes_from_environ = os.environ.get("TEST_DTYPES", "float16,float32,float64").split(
    ","
)
TEST_DTYPES = [getattr(torch, x) for x in dtypes_from_environ]

In [2]:
# embeddings
batch_size = 6
hidden_dim = 13

embeddings = torch.randn((batch_size, hidden_dim))
print(embeddings)

tensor([[ 0.4062, -0.8911, -1.2310,  0.9125, -1.5183,  0.8626,  1.1209,  0.9120,
         -0.0450,  0.0820, -0.7680, -0.7216,  0.3709],
        [ 0.9646,  0.7243,  0.1546, -0.4211, -0.1617, -1.6971, -0.2457,  1.7273,
          0.3448, -2.0385, -0.0436,  0.1245,  0.9833],
        [-0.8108, -1.5350, -0.0390,  1.8846, -1.1494, -1.5596, -1.0899, -2.0461,
          1.4187, -0.7099,  0.0949,  1.2315,  0.5286],
        [ 0.4251,  0.9468,  0.0475, -0.5334, -2.0651,  0.8345,  0.5650, -0.7073,
         -0.8625, -0.7808, -1.4542, -0.5173,  0.0982],
        [-2.8500,  0.5913, -2.4303, -0.1210, -1.3269, -2.2679, -0.3523,  0.0950,
          0.2033, -0.4219, -0.9945, -0.4709, -2.3443],
        [ 0.6186, -0.0486, -1.3054, -1.8398, -1.3805,  1.3874,  0.0744, -0.9763,
         -0.7732,  0.7235,  1.2201,  0.3298, -0.0446]])


In [3]:
# labels
labels = torch.LongTensor([0, 0, 0, 1, 1, 2])

In [4]:
# loss_funcA
temperature = 0.1

loss_funcA = NTXentLoss(temperature=temperature, )
print(loss_funcA)


NTXentLoss(
  (distance): CosineSimilarity()
  (reducer): MeanReducer()
)


In [5]:
# 计算 loss A
loss_A = loss_funcA(embeddings, labels)
print(loss_A)

tensor(2.9303)


In [6]:
# 复现计算过程
# 注意：distance默认是 CosineSimilarity()

pos_pairs = [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (3, 4), (4, 3)]
neg_pairs = [
    (0, 3), (0, 4), (0, 5),
    (1, 3), (1, 4), (1, 5),
    (2, 3), (2, 4), (2, 5),
    (3, 0), (3, 1), (3, 2), (3, 5),
    (4, 0), (4, 1), (4, 2), (4, 5),
    (5, 0), (5, 1), (5, 2), (5, 3), (5, 4),
]

total_lossA = 0

for a1, p in pos_pairs:
    # a1: anchor;
    # p: positive instance
    
    anchor, positive = embeddings[a1], embeddings[p]
    numeratorA = torch.exp(torch.matmul(anchor, positive) / (temperature * torch.norm(anchor) * torch.norm(positive)))
    
    denominatorA = numeratorA.clone()
    
    for a2, n in pos_pairs + neg_pairs:
        # n: negative instance
        
        if a2 == a1:
            negative = embeddings[n]
            curr_denomD = torch.exp(
                torch.matmul(anchor, negative) / (temperature * torch.norm(anchor) * torch.norm(negative))
            )
            if (a2, n) not in pos_pairs:
                denominatorA += curr_denomD
        else:
            continue
    
    print(numeratorA, denominatorA, - torch.log(numeratorA / denominatorA))
    curr_lossA = -torch.log(numeratorA / denominatorA)
    total_lossA += curr_lossA

total_lossA /= len(pos_pairs)

print("total_lossA: ", total_lossA)

rtol = 1e-4
assert torch.isclose(loss_A, total_lossA, rtol=rtol)

tensor(0.5735) tensor(62.2873) tensor(4.6878)
tensor(0.5758) tensor(62.2896) tensor(4.6838)
tensor(0.5735) tensor(3.0223) tensor(1.6621)
tensor(0.7222) tensor(3.1710) tensor(1.4795)
tensor(0.5758) tensor(9.7488) tensor(2.8291)
tensor(0.7222) tensor(9.8952) tensor(2.6175)
tensor(2.3383) tensor(88.9860) tensor(3.6390)
tensor(2.3383) tensor(14.7796) tensor(1.8438)
total_lossA:  tensor(2.9303)
